In [3]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np

mat_data_set = [()]
# Load MATLAB file
mat_file = r'C:\Device5_ 1.mat' 
mat_data = scipy.io.loadmat(mat_file)
mat_data_set = np.append(mat_data_set, mat_data)

mat_file = r'C:\Device2  1.mat' 
mat_data = scipy.io.loadmat(mat_file)
mat_data_set = np.append(mat_data_set, mat_data)

mat_file = r'C:\Device3  1.mat' 
mat_data = scipy.io.loadmat(mat_file)
mat_data_set = np.append(mat_data_set, mat_data)

mat_file = r'C:\Device4  1.mat' 
mat_data = scipy.io.loadmat(mat_file)
mat_data_set = np.append(mat_data_set, mat_data)

#j=0 dev5
#j=1 dev2
#j=2 dev3
#j=3 dev4

ce_window_smooth = [None] * len(mat_data_set)

for j in range(len(mat_data_set)):
    measurement = mat_data_set[j]['measurement']
    transient = measurement['transient']
    
    total_time = 0
    new_time = 0
    original = 0 
    time = np.array([])
    ce_data_list = []
    
    for i in range(len(transient[0,0][0])):
        date_str = transient[0,0][0][i][0][0]
        time_str = date_str.split(' ')[1]
        hour, minute, second = map(int, time_str.split(':'))
        
        new_time = hour*3600 + minute*60 + second
        if i == 0:
            original = new_time
        total_time = new_time - original
    
        time = np.append(time, total_time)
        
        ce_data = transient[0,0][0][i][2][0][0][3][0]
        ce_data_list.append(ce_data)
    
    ce_data_array = np.array(ce_data_list) 
    
    ce_sampling_data = [()]
    for i in range(len(ce_data_array)):
        if time[i]>=2000 and np.min(ce_data_array[i])<=4.0:
            break
        ce_sampling_data = np.append(ce_sampling_data, np.average(ce_data_array[i]))
    
    def calculate_weighted_average(x, t, span=15):
        theta = 2 / (span + 1)
        numerator = 0
        denominator = 0
    
        for i in range(t + 1):
            weight = (1 - theta) ** (t - i)
            numerator += weight * x[i]
            denominator += weight
    
        yt = numerator / denominator
        return yt
    
    span = 15
    ce_window_smooth[j] = np.zeros(len(ce_sampling_data))
    for t in range(len(ce_sampling_data)):
        ce_window_smooth[j][t] = calculate_weighted_average(ce_sampling_data, t, span)

print("Done")

Done


In [4]:
for q in range(len(mat_data_set)):
    len_ori = len(ce_window_smooth[q])
    print(len_ori)

103
106
96
91


In [5]:
ce_window_smooth_new = [None]*len(mat_data_set)
len_larger = len(ce_window_smooth[3])
for q in range(len(mat_data_set)):
    ce_window_smooth_new[q] = ce_window_smooth[q][0:len_larger]
    #print(len(output_array_new[q])-len(ce_window_smooth[q])

In [6]:
for q in range(len(mat_data_set)):
    len_ori = len(ce_window_smooth_new[q])
    print(len_ori)

91
91
91
91


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the CNN + LSTM model without MaxPool1d
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        # Define CNN layer (adjust kernel_size and padding)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)  # Convolutional Layer
        self.relu = nn.ReLU()  # ReLU activation
        # Define LSTM layer
        self.lstm = nn.LSTM(input_size=16, hidden_size=32, num_layers=1, batch_first=True)
        # Fully connected layer after LSTM
        self.fc1 = nn.Linear(32, 1)

    def forward(self, x):
        #print(f"Input shape: {x.shape}")
        x = x.unsqueeze(1)  # Add channel dimension (1 channel in this case for CNN)
        x = self.conv1(x)  # Convolution operation
       # print(f"After Conv1D shape: {x.shape}")
        x = self.relu(x)  # Apply ReLU activation
        # x = self.pool(x)  # Removed MaxPooling
        x = x.permute(0, 2, 1)  # Rearrange for LSTM (batch, seq_len, input_size)
        #print(f"After permute shape: {x.shape}")
        x, _ = self.lstm(x)  # Apply LSTM
        #print(f"After LSTM shape: {x.shape}")
        x = x[:, -1, :]  # Use the output of the last LSTM timestep
        x = self.fc1(x)  # Fully connected layer to get final output
        return x

# Instantiate the model
model = CNN_LSTM()

# Define loss and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop (element-wise training)
num_epochs = 100

for q in range(len(mat_data_set)):
    model.train()
    if q in [1, 2, 3]:
        data = ce_window_smooth[q]
        for epoch in range(num_epochs):
            for i in range(len(data)):            
                inputs = torch.tensor(data[i], dtype=torch.float32)  # Convert numpy array to tensor
                labels = torch.tensor(data[i], dtype=torch.float32)  # Convert numpy array to tensor
                
                # Zero the gradients
                optimizer.zero_grad()
        
                # Forward pass
                output = model(input_data)
        
                # Calculate loss
                loss = criterion(output, target.unsqueeze(0))
        
                # Backward pass and optimization
                loss.backward()
                optimizer.step()
    

# Test the model on the test data (element-wise)
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    test_output = []
    data_test = ce_window_smooth[0]
    for i in range(len(data_test)):
        test_input = data_test[i].unsqueeze(0)  # Input for testing (one element at a time)
        output = model(test_input.unsqueeze(0))
        test_output.append(output.item())
    print(f'Test output: {test_output}')


AttributeError: 'numpy.float64' object has no attribute 'unsqueeze'

In [ ]:
# Plot results
plt.plot(test_output, label='Predicted')
plt.plot(ce_window_smooth_new[0], label='Actual')
plt.legend()
plt.show()

In [ ]:
import numpy as np

# Function to calculate MSE
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Function to calculate R² (Coefficient of Determination)
def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)  # Residual sum of squares
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # Total sum of squares
    return 1 - (ss_res / ss_tot)

# Function to calculate MAE
def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

print("Done")

In [ ]:
y_true = ce_window_smooth_new[0]

y_pred = cnnlstm_out
# Calculate MSE
mse_cnnlstm = mean_squared_error(y_true, y_pred)

# Calculate R²
r2_cnnlstm = r2_score(y_true, y_pred)*100

# Calculate MAE
mae_cnnlstm = mean_absolute_error(y_true, y_pred)

print(mse_cnnlstm)
print(r2_cnnlstm)
print(mae_cnnlstm)